In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn import decomposition
import matplotlib.pyplot as plt
import DataHelper

e = DataHelper.eclass_data()

In [2]:
e.get_intro_pre(unique_students=True)

,index,anon_student_id,survey_id,duration,q01a,q01b,q27a,q27b,q03a,q03b,...,q04b,q14a,q14b,q29a,q29b,q07a,q07b,q31a,q31b,ResponseId
0,19,11542,00STAnSs9Sqb8sB,234.0,5.0,5.0,5.0,5.0,3.0,5.0,...,5.0,3.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,R_1I5iowCdb6VvSeo
1,20,11543,00STAnSs9Sqb8sB,688.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,R_1I5iowCdb6VvSeo
2,21,11544,00STAnSs9Sqb8sB,298.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,R_1I5iowCdb6VvSeo
3,22,11545,00STAnSs9Sqb8sB,1073.0,5.0,5.0,5.0,5.0,5.0,5.0,...,3.0,3.0,5.0,5.0,5.0,3.0,1.0,5.0,5.0,R_1I5iowCdb6VvSeo
4,23,11546,00STAnSs9Sqb8sB,427.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,3.0,5.0,5.0,5.0,1.0,5.0,5.0,5.0,R_1I5iowCdb6VvSeo
5,24,11547,00STAnSs9Sqb8sB,713.0,5.0,5.0,3.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,1.0,5.0,5.0,5.0,R_1I5iowCdb6VvSeo
6,25,11548,00STAnSs9Sqb8sB,1892.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,3.0,3.0,5.0,5.0,R_1I5iowCdb6VvSeo
7,26,11549,00STAnSs9Sqb8sB,377.0,5.0,5.0,3.0,5.0,1.0,1.0,...,5.0,1.0,5.0,1.0,1.0,3.0,3.0,5.0,5.0,R_1I5iowCdb6VvSeo
8,27,11550,00STAnSs9Sqb8sB,317.0,5.0,5.0,5.0,5.0,1.0,1.0,...,5.0,5.0,5.0,5.0,5.0,1.0,5.0,5.0,5.0,R_1I5iowCdb6VvSeo
9,28,11551,00STAnSs9Sqb8sB,912.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,3.0,5.0,5.0,5.0,R_1I5iowCdb6VvSeo


# Figure 1 : Cumulative number of student responses to ECLASS

# Table 1: Summary statistics describing differences in responses between intro and bfy

# Table 2: Demographic responses for ECLASS data set`

# Figure 2: plot comparing response rates per survey

# Table 3: Institutional descriptors table

# Table 4: Table on course information survey

# Figure 3: Spearman corrleations for ECLASS survey

# Figure 4: Skree plot for PCA of ECLASS survey

# Figure 5: Eigenvalues of first three principal components